In [ ]:
pip install pandas

In [ ]:
conda install matplotlib

In [ ]:
pip install openpyxl

In [ ]:
import datetime
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

kf_time_SV_initial = pd.read_excel("время св на точках.xlsx")
kf_time_SV_data_initial = pd.read_excel("отчет для расчета.xlsx")
cookers_initial = pd.read_excel("кол-во поваров на точке.xlsx")
df_concat_SV = pd.DataFrame()

#------------------------------
def samovivoz(x):
    index_iloc=0
    for i in range(kf_time_SV['время Minutes'].count()):
        if (kf_time_SV['metka'].iloc[i]==False) and \
        x['время открытия Minutes'] >= kf_time_SV['время Minutes'].iloc[i] and \
        x['время открытия Minutes'] < kf_time_SV['время Minutes'].iloc[i+1] and \
        x['Время открытия'].month == kf_time_SV['дата'].iloc[i].month and \
        x['Время открытия'].year == kf_time_SV['дата'].iloc[i].year and \
        x['Время открытия'].day == kf_time_SV['дата'].iloc[i].day:
            return int(i)
        elif (kf_time_SV['metka'].iloc[i]==True) and \
        x['время открытия Minutes'] >= kf_time_SV['время Minutes'].iloc[i] and \
        x['Время открытия'].month == kf_time_SV['дата'].iloc[i].month and \
        x['Время открытия'].year == kf_time_SV['дата'].iloc[i].year and \
        x['Время открытия'].day == kf_time_SV['дата'].iloc[i].day:
            return int(i)
        else:
            continue

#------------------------------                  
def max_time(x):
    time_pizza = 0
    time_goriachka = 0
    time_rolly = 0
    
    if (x['пиццы']==0 or pd.isna(x['пиццы'])):
        time_pizza = 0
    else:
        time_pizza = kf_time_SV['пиццы'].iloc[x['new']]
    
    if (x['горячка']==0 or pd.isna(x['горячка'])):
        time_goriachka = 0
        
    else:
        time_goriachka = kf_time_SV['горячка'].iloc[x['new']]
        
    if (x['роллы']==0 or pd.isna(x['роллы'])):
        time_rolly = 0
    else:
        time_rolly = kf_time_SV['роллы'].iloc[x['new']]
       
    number_list=[time_pizza, time_goriachka, time_rolly]
    #print(number_list)
    max_value = max(number_list)
    max_index = number_list.index(max_value)
    #print(max_index)
   
    if max_index==0:
        return time_pizza
    
    if max_index==1:
        return time_goriachka
    
    if max_index==2:
        return time_rolly

#------------------------------

Tpoints=['Алроса', 'Горького', 'Раздольная']
for i in Tpoints:
    
    kf_time_SV = kf_time_SV_initial[(kf_time_SV_initial['точка'] == i)]
    kf_time_SV_data = kf_time_SV_data_initial[(kf_time_SV_data_initial['Торговое предприятие'] == i)]
    cookers = cookers_initial[(cookers_initial['точка'] == i)]                       
    
    kf_time_SV_data.reset_index(drop=True)
    kf_time_SV = kf_time_SV.reset_index(drop=True)
    cookers = cookers.reset_index(drop=True)
    
    kf_time_SV['время Minutes'] = kf_time_SV.apply(lambda x: 60*x['время'].hour + x['время'].minute, axis=1)
    kf_time_SV['дата']=kf_time_SV.apply(lambda x: x['дата'].date(), axis=1)
    kf_time_SV['metka']=False

    #------------------------------
   
    for i in range(kf_time_SV['время Minutes'].count()-1):
        if kf_time_SV['дата'].iloc[i]<kf_time_SV['дата'].iloc[i+1]:
            kf_time_SV['metka'].iloc[i]=True

    #------------------------------   
    kf_time_SV['metka'].iloc[kf_time_SV['время Minutes'].count()-1]=True
    kf_time_SV_data['время открытия Minutes'] = kf_time_SV_data.apply(lambda x: 60*x['Время открытия'].hour + x['Время открытия'].minute + x['Время открытия'].second/60, axis=1)
    kf_time_SV_data['new'] = kf_time_SV_data.apply(samovivoz, axis=1)
    kf_time_SV_data['время заявленное кухней'] = kf_time_SV_data.apply(max_time, axis=1)

    cookers['Hour'] = cookers.apply(lambda x: x['час'].hour, axis=1)
    cookers['Date'] = cookers['дата '].dt.day
    cookers['Month'] = cookers['дата '].dt.month
    kf_time_SV_data['Date']=kf_time_SV_data['Время открытия'].dt.day
    kf_time_SV_data['Month']=kf_time_SV_data['Время открытия'].dt.month
    kf_time_SV_data['Hour']=kf_time_SV_data['Время открытия'].dt.hour
    kf_time_SV_data.head()
    kf_time_SV_data['Количество поваров'] = 0
   
    
    for i in range(cookers.shape[0]):
        kf_time_SV_data.loc[(kf_time_SV_data['Hour'] == cookers['Hour'][i]) & (kf_time_SV_data['Month'] == cookers['Month'][i]) & (kf_time_SV_data['Date'] == cookers['Date'][i]), 'Количество поваров'] = cookers['кол-во поваров'][i]

    df_concat_SV = pd.concat([df_concat_SV, kf_time_SV_data], axis=0)

df_concat_SV.to_excel('itog.xlsx')

In [ ]:
import datetime 
from datetime import date, datetime, time, timedelta
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
kf_logostic = pd.read_excel("itog.xlsx")

def f(x):
    if (x['Опоздание доставка общ'] >= 0):
        return 0
    else:
        return 1
    
def f1(x):
    if (x['Опоздание кухня'] >= 0):
        return 0
    else:
        return 1
    
def f2(x):
    if (x['Опоздание кухня'] < 0):
        return x['Опоздание доставка общ'] - x['Опоздание кухня']
    else:
        return x['Опоздание доставка общ']
    
    
def f3(x):
    if (x['Опоздание только курьеры'] >= 0):
        return 0
    else:
        return 1 

def f4(x):
    if pd.isna(x['Источник заказа']):
        return 'kvikfud.ru'
    else:
        return x['Источник заказа'] 

kf_logostic['Время доставки'] = kf_logostic['Фактическое время доставки'] - kf_logostic['Время открытия']
kf_logostic['Время доставки'] = kf_logostic['Время доставки'].dt.total_seconds()/60

kf_logostic['Опоздание доставка общ'] = kf_logostic['Планируемое время доставки'] - kf_logostic['Фактическое время доставки']
kf_logostic['Опоздание доставка общ'] = kf_logostic['Опоздание доставка общ'].dt.total_seconds()/60

kf_logostic['озвучено время курьерами'] = kf_logostic['Планируемое время доставки'] - kf_logostic['Время открытия']
kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].dt.total_seconds()/60 - kf_logostic['время заявленное кухней']

kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].round(decimals=-1)

kf_logostic['Флаг опоздания общ'] = kf_logostic.apply(f, axis=1)
kf_logostic['Флаг опоздания общ'].value_counts()

kf_logostic['Время приготовления'] = kf_logostic['Время печати накладной'] - kf_logostic['Время открытия']
kf_logostic['Время приготовления'] = kf_logostic['Время приготовления'].dt.total_seconds()/60

kf_logostic['Опоздание кухня'] = kf_logostic['время заявленное кухней']- kf_logostic['Время приготовления']

kf_logostic['Флаг опоздания кухня'] = kf_logostic.apply(f1, axis=1)
kf_logostic['Флаг опоздания кухня'].value_counts()

kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)

kf_deep= kf_logostic.copy()

kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['Тип заказа'] == 'На время самовывоз'].index)
kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['Тип заказа'] == 'На время курьером'].index)

print('Доставлено заказов вовремя: ', kf_logostic['Флаг опоздания общ'].value_counts()[0])
print('Доставлено заказов с опозданием: ', kf_logostic['Флаг опоздания общ'].value_counts()[1])
print('Процент опозданий заказов: ', round(100*kf_logostic['Флаг опоздания общ'].value_counts()[1]/(kf_logostic['Флаг опоздания общ'].value_counts()[1]+kf_logostic['Флаг опоздания общ'].value_counts()[0]),2),'%')
print('\n')

print('Приготовлено вовремя заказов: ', kf_logostic['Флаг опоздания кухня'].value_counts()[0])
print('Опозданий в приготовлении: ', kf_logostic['Флаг опоздания кухня'].value_counts()[1])
print('Процент опозданий в приготовлении: ', round(100*kf_logostic['Флаг опоздания кухня'].value_counts()[1]/(kf_logostic['Флаг опоздания кухня'].value_counts()[1]+kf_logostic['Флаг опоздания кухня'].value_counts()[0]),2),'%')
print('\n')

kf_logostic_without_bad_closed = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)
kf_logostic_without_bad_closed = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['Тип заказа'] == 'Доставка самовывоз'].index)
kf_logostic_without_bad_closed['Опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f2, axis=1)
kf_logostic_without_bad_closed['Флаг опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f3, axis=1)

print('Доставлено курьерами вовремя: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0])
print('Опозданий по вине курьеров: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1])
print('Процент опозданий по вине курьеров: ', round(100*kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]/(kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]+kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0]),2),'%')
print('\n')

#kf_logostic_without_bad_closed['время заявленное кухней'].value_counts()
kf_logostic_without_bad_closed['Источник заказа'] = kf_logostic_without_bad_closed.apply(f4, axis=1)
kf_logostic_without_delivery = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['Источник заказа'] == 'delivery-club.ru'].index)
print(kf_logostic_without_delivery.groupby('озвучено время курьерами')['Источник заказа'].value_counts())

kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)].head(10)

otchet=kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)]
otchet.to_excel('проблемные заказы.xlsx')

In [ ]:
kf_deep['Day of Week'] = kf_deep['Время открытия'].dt.dayofweek
kf_deep['Day of Week'].replace({0: "ПН", 1: "ВТ", 2: "СР", 3: "ЧТ", 4: "ПТ", 5: "СБ", 6: "ВС"}, inplace=True)
kf_deep['Hour'] = kf_deep['Время открытия'].dt.hour

kf_deep.loc[(kf_deep['Тип заказа'] == 'На время курьером'), 'Время открытия'] = kf_deep['Время печати накладной'].loc[(kf_deep['Тип заказа']\
            == 'Доставка самовывоз')]-timedelta(minutes=30)
kf_deep.loc[(kf_deep['Тип заказа'] == 'На время самовывоз'), 'Время открытия'] = kf_deep['Время печати накладной'].loc[(kf_deep['Тип заказа'] \
            == 'Доставка самовывоз')]-timedelta(minutes=30)

kf_deep['Время приготовления'] = kf_deep['Время печати накладной'] - kf_deep['Время открытия']
kf_deep['Время приготовления'] = kf_deep['Время приготовления'].dt.total_seconds()/60

def cooking_time(kf_deep_x, point, day, start, end):
    kf_deep = kf_deep_x.loc[(kf_deep_x['Торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end)] 
    return kf_deep['Время приготовления'].mean(), kf_deep['Время приготовления'].max(), kf_deep['Флаг опоздания кухня'].sum(), kf_deep['Флаг опоздания кухня'].count(), \
           kf_deep['общая сумма чека'].sum(), kf_deep['Количество поваров'].mean()

cooking_array = np.array([['Алроса', 'ПН', 12, 14],
                 ['Алроса', 'ВТ', 12, 14],
                 ['Алроса', 'СР', 12, 14],
                 ['Алроса', 'ЧТ', 12, 14],
                 ['Алроса', 'ПТ', 12, 14],
                 ['Алроса', 'СБ', 12, 14],
                 ['Алроса', 'ВС', 12, 14],
                 ['Алроса', 'ПН', 17, 21],
                 ['Алроса', 'ВТ', 17, 21],
                 ['Алроса', 'СР', 17, 21],
                 ['Алроса', 'ЧТ', 17, 21],
                 ['Алроса', 'ПТ', 17, 21],
                 ['Алроса', 'СБ', 17, 21],
                 ['Алроса', 'ВС', 17, 21],
                 ['Горького', 'ПН', 12, 14],
                 ['Горького', 'ВТ', 12, 14],
                 ['Горького', 'СР', 12, 14],
                 ['Горького', 'ЧТ', 12, 14],
                 ['Горького', 'ПТ', 12, 14],
                 ['Горького', 'СБ', 12, 14],
                 ['Горького', 'ВС', 12, 14],
                 ['Горького', 'ПН', 17, 21],
                 ['Горького', 'ВТ', 17, 21],
                 ['Горького', 'СР', 17, 21],
                 ['Горького', 'ЧТ', 17, 21],
                 ['Горького', 'ПТ', 17, 21],
                 ['Горького', 'СБ', 17, 21],
                 ['Горького', 'ВС', 17, 21],
                 ['Раздольная', 'ПН', 12, 14],
                 ['Раздольная', 'ВТ', 12, 14],
                 ['Раздольная', 'СР', 12, 14],
                 ['Раздольная', 'ЧТ', 12, 14],
                 ['Раздольная', 'ПТ', 12, 14],
                 ['Раздольная', 'СБ', 12, 14],
                 ['Раздольная', 'ВС', 12, 14],
                 ['Раздольная', 'ПН', 17, 21],
                 ['Раздольная', 'ВТ', 17, 21],
                 ['Раздольная', 'СР', 17, 21],
                 ['Раздольная', 'ЧТ', 17, 21],
                 ['Раздольная', 'ПТ', 17, 21],
                 ['Раздольная', 'СБ', 17, 21],
                 ['Раздольная', 'ВС', 17, 21]])

label = []
max_array = []
min_array = []
mean_array = []
lateness_array = []
receipts_array = []
ours_array = []
sum_array = []
cookers_array = []
for i in cooking_array:
    mean, max, lateness, receipts, sum, cookers = cooking_time(kf_deep, str(i[0]), str(i[1]), int(i[2]), int(i[3]))
    label.append(str(i[0]) + ' ' + str(i[2]) + '-' + str(i[3]) + ' ' + str(i[1]))
    ours_array.append(int(i[3])-int(i[2]))
    mean_array.append(int(mean))
    max_array.append(int(max))
    lateness_array.append(lateness)
    receipts_array.append(receipts)
    sum_array.append(sum)
    cookers_array.append(cookers)
    
df = pd.DataFrame(data=mean_array, index=label) 
df.columns =['Среднее время']
df['Максимальное'] = max_array
df['Опоздания кухни'] = lateness_array
df['Чеков всего'] = receipts_array
df['Сумма чеков'] = sum_array
df['Чеков в час'] = np.array(receipts_array)/np.array(ours_array)
df['Чеков в час'] = df['Чеков в час'].round(decimals = 0).astype(int)
df['Сумма в час'] = np.array(sum_array)/np.array(ours_array)
df['Сумма в час'] = df['Сумма в час'].round(decimals = 0).astype(int)
df['Количество поваров'] = cookers_array
df['Сумма на повара в час'] = df['Сумма в час']/cookers_array
df['Сумма на повара в час']=df['Сумма на повара в час'].round(decimals = 0).astype(int)
df['Количество поваров'] = df['Количество поваров'].round(decimals = 1).astype(int)
display(df.sort_values(by=['Среднее время'], ascending=False).style.background_gradient(cmap='Blues'))

kf_deep = kf_deep.drop(kf_deep[kf_deep['Тип заказа'] == 'На время самовывоз'].index)
kf_deep = kf_deep.drop(kf_deep[kf_deep['Тип заказа'] == 'Доставка самовывоз'].index)

kf_deep['Стоял на точке'] = kf_deep['Время отправки доставки'] - kf_deep['Время печати накладной'] 
kf_deep['Стоял на точке'] = kf_deep['Стоял на точке'].dt.total_seconds()/60
bins = [0, 10, 20, 30, 40, 50, 60, 1000]
labels = ['<10', '10-20', '20-30', '30-40', '40-50', '50-60', '>60']
cats = pd.cut(kf_deep['Стоял на точке'], bins, labels=labels)
pd.value_counts(cats)
kf_deep['Bins']=cats

def bins(kf_deep_x, point):
    kf_deep = kf_deep_x.loc[kf_deep_x['Торговое предприятие'] == point] 
    return kf_deep['Bins'].value_counts()  

def bins_day(kf_deep_x, point, day, start, end):
    kf_deep = kf_deep_x.loc[(kf_deep_x['Торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end)] 
    return kf_deep['Bins'].value_counts()

def table_day(kf_deep_x, point="всем точкам"):
    if point=="всем точкам":
        kf_deep = kf_deep_x
    else:
        kf_deep = kf_deep_x.loc[(kf_deep_x['Торговое предприятие'] == point)]
    print('Данные по', point, 'Сколько заказов ожидали отправки')
    grouped = kf_deep.groupby('Day of Week').Bins.value_counts().unstack(0)[["ПН", "ВТ", "СР", "ЧТ", "ПТ", "СБ", "ВС"]].fillna(0.0).astype(int)
    print(grouped[3:], '\n')
#    print(kf_deep.head(), '\n')
    grouped[3:].plot(kind='bar', title = "Данные по " + point, xlabel='Время ожидания на точке', ylabel='Количество заказов').get_figure().savefig(str(point)+'.png')
    return kf_deep['Bins'].value_counts()

final_table_amount_orders = pd.DataFrame(index=['<10', '10-20', '20-30', '30-40', '40-50', '50-60', '>60'])
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Алроса'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Алроса', "ПТ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Алроса', "СБ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр Sat 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Горького'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Горького', "ПТ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Горького', "СБ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор Sat 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Раздольная'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Раздольная', "ПТ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Раздольная', "СБ", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд Sat 17-20'})

table_day(kf_deep)
table_day(kf_deep, 'Алроса')
table_day(kf_deep, 'Горького')
table_day(kf_deep, 'Раздольная')

print(final_table_amount_orders)

In [ ]:
import datetime 
from datetime import date, datetime, time, timedelta
import pandas as pd
import numpy as np 
pd.set_option("display.max_rows", None,'display.max_columns', None)
pd.set_option('display.max_rows', None)
deliveryman = pd.read_excel("itog.xlsx")

deliveryman.head()
deliveryman['Курьер'].unique()
deliveryman['Курьер'].value_counts()
deliveryman['День'] = deliveryman['Время открытия'].dt.dayofweek 
deliveryman['День'].replace({0: "ПН", 1: "ВТ", 2: "СР", 3: "ЧТ", 4: "ПТ", 5: "СБ", 6: "ВС"}, inplace=True)
deliveryman['Время'] = deliveryman['Время отправки доставки'].dt.strftime('%H:%M')

def order_amount(x):
    a = deliveryman.loc[(deliveryman['Курьер'] == x['Курьер']) & (deliveryman['Время отправки доставки'] > (x['Время отправки доставки']) - timedelta(minutes=5)) \
                        & (deliveryman['Время отправки доставки'] < (x['Время отправки доставки']) + timedelta(minutes=5)) \
                        & (deliveryman['День']==x['День']) & (deliveryman['Торговое предприятие']==x['Торговое предприятие'])].shape[0]
    return a

df_concat = pd.DataFrame(columns=['Время отправки доставки','Курьер', 'Заказов в рейс'])

for dm in deliveryman['Курьер'].unique():
    deliveryman['Заказов в рейс'] = deliveryman.loc[(deliveryman['Курьер'] == dm)].apply(order_amount, axis=1)
    df_concat = pd.concat([df_concat, deliveryman.loc[(deliveryman['Курьер'] == dm)]], axis=0)

for_print = df_concat[df_concat['Заказов в рейс'] > 3].sort_values(by=['Заказов в рейс'], ascending=False).sort_values(by=['Время отправки доставки'], ascending=False)[['Курьер', 'Заказов в рейс','Время','Торговое предприятие','День']]


#print(for_print.sort_values(by=['Торговое предприятие'], ascending=False).sort_values(by=['Время'], ascending=False))
print(for_print[for_print['Торговое предприятие']=='Алроса'].sort_values(by=['Время'], ascending=False).reset_index(drop=True), '\n')
print(for_print[for_print['Торговое предприятие']=='Горького'].sort_values(by=['Время'], ascending=False).reset_index(drop=True), '\n')
print(for_print[for_print['Торговое предприятие']=='Раздольная'].sort_values(by=['Время'], ascending=False).reset_index(drop=True), '\n')

df_concat[df_concat['Заказов в рейс'] > 3].sort_values(by=['Заказов в рейс'], ascending=False).sort_values(by=['Время отправки доставки'], ascending=False).to_excel('рейсы.xlsx')


pivot_table_DM = pd.pivot_table(df_concat[['Date', 'День', 'Торговое предприятие','Курьер']], index = ['Курьер','День', 'Торговое предприятие'], aggfunc='count')
print(pivot_table_DM)

orders = df_concat.groupby('Курьер')['Заказов в рейс'].value_counts()
orders= orders.to_frame()
orders=orders.rename(columns={"Заказов в рейс": "Количество рейсов"})
orders=orders.reset_index()
orders=orders.rename(columns={"Заказов в рейс": "Заказов в рейсе"})
orders['Количество рейсов'] = orders['Количество рейсов']/orders['Заказов в рейсе']
orders = orders.astype({'Количество рейсов': 'int32','Заказов в рейсе': 'int32'})

print(orders.sort_values(by=['Заказов в рейсе'], ascending=False))
full_table = orders.groupby('Заказов в рейсе')['Количество рейсов'].sum().sort_index(ascending=False)
full_table=full_table.to_frame()
full_table=full_table.reset_index()
full_table